importとか諸々

In [39]:
import pandas as pd
import numpy as  np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

データの読み込み

In [40]:
test = pd.read_csv('test.csv')
year1 = pd.read_csv('year1.csv')
year2 = pd.read_csv('year2.csv')
year3 = pd.read_csv('year2.csv')
year4 = pd.read_csv('year2.csv')

欠損値を0こ以上含むデータの確認

In [41]:
print(test.isnull().sum()[test.isnull().sum()>0].sort_values(ascending=False))
print(year1.isnull().sum()[year1.isnull().sum()>0].sort_values(ascending=False))
print(year2.isnull().sum()[year2.isnull().sum()>0].sort_values(ascending=False))
print(year3.isnull().sum()[year3.isnull().sum()>0].sort_values(ascending=False))
print(year4.isnull().sum()[year4.isnull().sum()>0].sort_values(ascending=False))

Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)


データをまとめる

In [42]:
all_data = pd.concat([year1,year2,year3,year4])

？をその行の平均に痴漢
後に要考慮

In [43]:
test = test.replace("?", test.mean())

In [44]:
all_data = all_data.replace("?",all_data.mean())

target とidをのぞいたcolを用意

In [49]:
num_cols = [i for i in all_data.columns if i not in all_data and i not in ['TARGET','id']]

In [ ]:
Xとyを決定

In [50]:
X = all_data.loc[:,num_cols]

In [51]:
y = all_data['TARGET']

データを分割

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y,test_size=0.3,random_state=0)